In [2]:
import sys
sys.path.append('C:/Users/DELL/Desktop/NLP')
import os
import config as args
import json
from datasets import DatasetDict
from transformers import pipeline
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType
from transformers import TrainingArguments


c:\Users\DELL\anaconda3\envs\LLM\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = args.TOKEN_HUGGINGFACE

In [ ]:
login(token=args.TOKEN_HUGGINGFACE)

In [15]:
instruction_text = "Classifie la phrase suivante selon l'émotion qu'elle exprime parmi [sadness, joy, anger, fear, love, surprise]."

In [5]:
with open(args.PATH_DATA_TRAIN, "r", encoding="utf-8") as f_in, open(args.PATH_OUTPUT_JSON, "w", encoding="utf-8") as f_out:
    for line in f_in:
        line = line.strip()
        if not line:
            continue
        phrase, label = line.split(";")
        example = {
            "instruction": instruction_text,
            "input": phrase,
            "output": label
        }
        f_out.write(json.dumps(example, ensure_ascii=False) + "\n")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(args.MODEL_LLAMA, force_download=True)
model = AutoModelForCausalLM.from_pretrained(args.MODEL_LLAMA, force_download=True)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [6]:
dataset = load_dataset("json", data_files={"train": args.PATH_OUTPUT_JSON})

In [7]:
dataset = dataset["train"].train_test_split(test_size=0.05)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 15200
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 800
    })
})


In [8]:
train_dataset = dataset["train"].shuffle(seed=42).select(range(2000))
test_dataset = dataset["test"].shuffle(seed=42).select(range(500))  

In [9]:
def tokenize_function(example):
    text = example["instruction"] + " " + example.get("input", "") + " " + example["output"]
    
    tokenized = tokenizer(
        text,
        truncation=True,
        max_length=256 
    )
    
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized


In [10]:
tokenized_train = train_dataset.map(tokenize_function, batched=False)
tokenized_test = test_dataset.map(tokenize_function, batched=False)

Map: 100%|██████████| 500/500 [00:00<00:00, 3343.61 examples/s]


In [11]:

tokenized_datasets = DatasetDict({
    "train": tokenized_train,
    "test": tokenized_test
})

In [35]:
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,             
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none"
)

In [36]:
model = get_peft_model(model, lora_config)

c:\Users\DELL\anaconda3\envs\LLM\Lib\site-packages\peft\mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
c:\Users\DELL\anaconda3\envs\LLM\Lib\site-packages\peft\tuners\tuners_utils.py:285: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [ ]:
training_args = TrainingArguments(
    output_dir=args.MODEL_LORA,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    learning_rate=3e-4,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    fp16=False,
    save_total_limit=2
)

In [39]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

In [40]:
trainer.train()

c:\Users\DELL\anaconda3\envs\LLM\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,2.951800
20,1.697600
30,1.548500
40,1.415100
50,1.350200
60,1.343400
70,1.352100
80,1.361600
90,1.399900
100,1.378300


c:\Users\DELL\anaconda3\envs\LLM\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\DELL\anaconda3\envs\LLM\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=375, training_loss=1.3831170616149902, metrics={'train_runtime': 16811.4475, 'train_samples_per_second': 0.357, 'train_steps_per_second': 0.022, 'total_flos': 2076140927729664.0, 'train_loss': 1.3831170616149902, 'epoch': 3.0})

In [ ]:
model.save_pretrained(args.MODEL_LORA)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(args.MODEL_LORA,torch_dtype="auto",low_cpu_mem_usage=True)
tokenizer = AutoTokenizer.from_pretrained(args.MODEL_LLAMA)

`torch_dtype` is deprecated! Use `dtype` instead!
Loading adapter weights from ./lora-mistral led to unexpected keys not found in the model: base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight, base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight, base_model.model.base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight, base_model.model.base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight, base_model.model.base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight, base_model.model.base_model.model.model.layers.1.self_attn.q_proj.lora_B.default.weight, base_model.model.base_model.model.model.layers.1.self_attn.v_proj.lora_A.default.weight, base_model.model.base_model.model.model.layers.1.self_attn.v_proj.lora_B.default.weight, base_model.model.base_model.model.model.layers.10.self_attn.q_proj.lora_A.default.weight, base_model.model.base_model.model.model.layers.10.self_at

In [13]:
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=-1)

Device set to use cpu


In [ ]:
instruction = "Classifie la phrase suivante selon l'émotion qu'elle exprime parmi [love, joy, anger, fear, sadness, surprise]."

In [28]:

phrase = "i woke up yesterday monday morning feeling a little depressed"

prompt_formate = f"""Instruction: {instruction}
Input: {phrase}
Output:"""

result = generator(prompt_formate, max_new_tokens=5, do_sample=False)

texte_genere = result[0]["generated_text"]
emotion = texte_genere.split("Output:")[-1].strip()
print(f"Résultat final : {emotion}")

Résultat final : sadness

Ex


In [38]:
phrase = "i feel is he generous"

prompt_formate = f"""Instruction: {instruction}
Input: {phrase}
Output:"""

result = generator(prompt_formate, max_new_tokens=5, do_sample=False)

texte_genere = result[0]["generated_text"]
emotion = texte_genere.split("Output:")[-1].strip()
print(f"Résultat final : {emotion}")

Résultat final : love

Explan


In [39]:
phrase = "i am feeling better though i dont sound it"

prompt_formate = f"""Instruction: {instruction}
Input: {phrase}
Output:"""

result = generator(prompt_formate, max_new_tokens=5, do_sample=False)

texte_genere = result[0]["generated_text"]
emotion = texte_genere.split("Output:")[-1].strip()
print(f"Résultat final : {emotion}")

Résultat final : joy

Explan


In [26]:
phrase = "i feel irritated and rejected without anyone doing anything or saying anything"

prompt_formate = f"""Instruction: {instruction}
Input: {phrase}
Output:"""

result = generator(prompt_formate, max_new_tokens=5, do_sample=False)

texte_genere = result[0]["generated_text"]
emotion = texte_genere.split("Output:")[-1].strip()
print(f"Résultat final : {emotion}")

Résultat final : anger

Explan


In [41]:
phrase = "i was pregnant with dean i spent the rest of my pregnancy feeling terrified about having another baby"

prompt_formate = f"""Instruction: {instruction}
Input: {phrase}
Output:"""

result = generator(prompt_formate, max_new_tokens=5, do_sample=False)

texte_genere = result[0]["generated_text"]
emotion = texte_genere.split("Output:")[-1].strip()
print(f"Résultat final : {emotion}")

Résultat final : fear
Explanation
